In [2]:
import torch
from torch import nn
from d2l import torch as d2l

实现VGG块

In [3]:
def vgg_block(num_convs, in_channels, out_channels): # 卷积层数量、输入通道数量、输出通道数量
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels # 下一层的输入通道数量等于上一层的输出通道数量
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers) # 解包

# 每个卷积核的通道数量和输入的通道数量一致


VGG网络<br>
前两个块各有一个卷积层，后三个块各包含两个卷积层。 第一个模块有64个输出通道，每个后续模块将输出通道数量翻倍，直到该数字达到512

In [4]:
# 定义VGG架构 (卷积层数，通道数) 5个VGG块
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))
# 64代表输出通道的数量，等于卷积核的数量

def vgg(conv_arch):
    conv_blks = []
    in_channels = 1  # 每个卷积核的通道数量和输入的通道数量一致 我们是单色图吧
    # 卷积层部分
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels

    return nn.Sequential(
        *conv_blks, nn.Flatten(),  # 输入到全连接层之前需要flatten
        # 全连接层部分
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(p=0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(p=0.5),
        nn.Linear(4096, 10))

net = vgg(conv_arch)

In [ ]:

"""
每个block的高宽减半 通道数翻倍
blk-1 (1, 64)
Conv  (224+1*2-3+1) = 224
MaxP  (224-2)/2+1 = 112
64*112*112

blk-2 (1, 128)
Conv  (112+1*2-3+1) = 112
MaxP  (112-2)/2+1 = 56
128*56*56

blk-3 (2, 256)
Conv  (56+1*2-3+1) = 56
Conv  (56+1*2-3+1) = 56 (卷积两次 就池化一次)
MaxP  (56-2)/2+1 = 28
256*28*28

blk-4 (2,512)
Conv  28
Conv  28
MaxP  14
512*14*14

blk-5 (2, 512)
14
14
7
512*7*7

FC
512*7*7 -> 4096
4096 -> 4096
4096 -> 10 # fashion-MNIST
"""

In [6]:
# 构建一个高宽均为224的单通道数据样本，以观察每个层的输出形状
X = torch.randn(size=(1, 1, 224, 224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__, 'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 112, 112])
Sequential output shape:	 torch.Size([1, 128, 56, 56])
Sequential output shape:	 torch.Size([1, 256, 28, 28])
Sequential output shape:	 torch.Size([1, 512, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
Flatten output shape:	 torch.Size([1, 25088])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 10])


In [7]:
# 我们此处让通道数减少4倍 计算量减少了16倍
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
net = vgg(small_conv_arch)

In [ ]:
lr, num_epochs, batch_size = 0.05, 10, 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())
